0) Just some imports

In [673]:
import re
import math
import numpy as np
import common as cm
from scipy import spatial

# Query Expansion

# 1) Simple search engine (for 3.0)

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [674]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        
        self.corM = [] ### a correlation matrix
        self.corM_A = [] ### helpful matrices
        self.corM_transposed = []
        self.corM_final = []
        
        self.idfs_dict = {}
        

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### How common is word across a set of documents
    ### IDF = log(N / df(w)); N - total number of documents, df(w) - frequency of documents containing word 'w'
    def computeIDFs(self, documents):
        
        self.idfs = []
        N = len(documents)
        df = []
        for i, term in enumerate(self.terms):
            df.append(0)
            for document in documents:
                found = False
                for token in document.tokens:
                    if token == term and not found:
                        df[i] += 1
                        found = True
                    
        for freq in df:
            self.idfs.append(math.log10(N / freq))
            
        for term, idf in zip(self.terms, self.idfs):
            self.idfs_dict[term] = idf 


    def computeCorM(self, documents):
        
        # TODO: terms are only those which are in dictionary (?)
        # COMPUTE A
        blank_row = [0]*len(documents)
        row_term_id_dict = {} 
        # for iter, term in enumerate(self.terms):
        #    row_term_id_dict[term] = iter
        #    self.corM_A.append(blank_row)
        
        # TODO: terms are only those which are in documents
        i = 0
        for document in documents:
            for token in document.tokens:
                if token not in row_term_id_dict:
                    row_term_id_dict[token] = i
                    self.corM_A.append(blank_row)
                    i += 1
        print(row_term_id_dict)
            
        # COMPUTE A -> look for term in all documents, if found, set -1 in a cell
        for document in documents:
            tokens = document.tokens
            update_count = 0
            for token in tokens:
                if token in row_term_id_dict:
                    self.corM_A[row_term_id_dict[token]][document.doc_id] += 1
                    update_count += 1
                else: 
                    print("Token not found in dictionary")
            
        # COMPUTE A-NORM -> TODO: divide by number of occurences in row (?)
        for i, row in enumerate(self.corM_A):
            sum = 0
            for el in row:
                sum += el
            
            for j in range(len(row)):
                self.corM_A[i][j] /= sum
        
        # COMPUTE A-NORM-TRANSPOSE
        for j in range(len(self.corM_A[0])):
            row = []
            for i in range(len(self.corM_A[0])):
                row.append(self.corM_A[i][j])
            self.corM_transposed.append(row)
            
        # COMPUTE CORRELATION MATRIX
        self.corM = np.dot(np.array(self.corM_A),np.array(self.corM_transposed))
                     
### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [675]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [676]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [677]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation
        
        self.bow_lenght = 0
        self.bow_dict = {}
        self.tf_dict = {}

    ### complete this method; it should compute a BOW representation
    def computeBOW_Representation(self):
        
        for term in self.dictionary.terms:
            self.bow_dict[term] = 0
            
        for token in self.tokens:
            if token in self.bow_dict:
                self.bow_dict[token] += 1
        
        self.bow = self.bow_dict.items()
        self.bow_lenght = len(self.bow)
        
        
    ### complete this method; it should compute a TF representation
    ### el: (number of times t occurs in d / total number of terms in d)
    def computeTF_Representation(self):
        denominator = self.bow_lenght
        for term in self.bow:
            self.tf.append(term[1] / denominator)
            self.tf_dict[term[0]] = term[1] / denominator
            
    ### complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    ### TFxIDF(t,d,D) = TF(t,d) * IDF(t,D)
    def computeTF_IDF_Representation(self):
        for term0, term1 in self.tf_dict.items():
            self.tf_idf.append(term1*self.dictionary.idfs_dict[term0])
                     
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]


1.4) Compute IDFs here

In [678]:
### COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])

# LEAST COMMON WORDS - HIGH IDF
print("Least common words: ", res[-5:], "\n")

# MOST COMMON WORDS - LOW IDF
print("Most common words: ", res[:5], "\n")

Least common words:  [['working', 1.9444826721501687], ['www', 1.9444826721501687], ['york', 1.9444826721501687], ['young', 1.9444826721501687], ['zdm', 1.9444826721501687]] 

Most common words:  [['learning', 0.009984220906600923], ['machine', 0.009984220906600923], ['the', 0.1962946451439682], ['and', 0.3212333817522682], ['description', 0.3881801713828814]] 



1.5) Compute the document representations (for each document run computeRepresentations())

In [679]:
for d in documents: 
    d.computeRepresentations()
    
### SOME DEBUG (you should see some 4s - which terms are these?)
#print(documents[0].bow)
for tuple in documents[0].bow:
    if tuple[1] == 4:
        print(tuple, end="\n")

('learning', 4)
('machine', 4)


1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [680]:
def getSimilarity(v1, v2):
    return 1 - spatial.distance.cosine(v1, v2)

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [681]:
query = "machine learning"
#query = "academic research"
#query = "international conference"
#query = "international conference washington"

In [682]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    # construct document which is made of query
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    
    # in each of documents check cosine similarity to query (here "machine learning")
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.017145022655810083 | DOC ID = 63
AI / Machine Learning Resources
 AI / Machine Learning Resources. General Machine Learning. The Journal
 of Machine Learning. MLnet Machine Learning Archive at GMD. The ... 


RANK = 2 WITH SIMILARITY = 0.012801913843626722 | DOC ID = 77
Machine Learning
 Machine Learning. Machine Learning Home Page (Editor) Machine Learning Home
 Page (Publisher) Machine Learning Online by Kluwer Academic Publishers: ... 


RANK = 3 WITH SIMILARITY = 0.012340729913628334 | DOC ID = 34
Machine Learning
 Machine Learning. Related Sites. Machine Learning Resources courtesy
 of David Aha A Machine Learning Tutorial a good overview of the ... 


RANK = 4 WITH SIMILARITY = 0.011069910022798668 | DOC ID = 81
Oxford University Machine Learning Group
 Machine Learning at the Computing Laboratory. ... Logic Programming and
 Learning and Intelligent Systems. Other Machine Learning Groups. ... 


RANK = 5 WITH SIMILARITY = 0.010455352603751189 | DOC ID

# 2) Query expansion

## 2.1) Correlation matrix (for 3.5)

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [683]:
### COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
for row in dictionary.corM:
    print(row, end="\n")

{'david': 0, 'aha': 1, 'machine': 2, 'learning': 3, 'page': 4, 'resource': 5, 'suggestion': 6, 'welcome': 7, 'wizrule': 8, 'zdm': 9, 'scientific': 10, 'ltd': 11, 'conference': 12, 'announcement': 13, 'course': 14, 'data': 15, 'repository': 16, 'description': 17, 'comprehensive': 18, 'from': 19, 'application': 20, 'tutorials': 21, 'home': 22, 'cbr': 23, 'folks': 24, 'and': 25, 'casebased': 26, 'reasoning': 27, 'all': 28, 'entry': 29, 'dedicated': 30, 'interface': 31, 'geographic': 32, 'location': 33, 'please': 34, 'send': 35, 'edit': 36, 'list': 37, 'for': 38, 'people': 39, 'uci': 40, 'the': 41, 'majority': 42, 'were': 43, 'contributed': 44, 'researcher': 45, 'outside': 46, 'database': 47, 'domain': 48, 'theory': 49, 'generator': 50, 'that': 51, 'are': 52, 'used': 53, 'group': 54, 'investigate': 55, 'mechanism': 56, 'which': 57, 'knowledge': 58, 'acquired': 59, 'through': 60, 'experience': 61, 'information': 62, 'mlnet': 63, 'ois': 64, 'find': 65, 'network': 66, 'online': 67, 'service':

2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [684]:
query = "machine"
#query = "algorithm"
# query = "learning"
# query = "conference"
# query = "research"
# query = "concept"

def suggestKeywords(query, dictionary):
    ### TODO
    print("Suggestions")
    pass
        
suggestKeywords(query, dictionary)

Suggestions


# 2.2) Rocchio algorithm (for 4.0)

- a method for updating a query vector - vector space model must be used to represent documents (TF-IDF vectors)

- linear combination of:
    * q
    * D_r
    * D_nr
    
- new q_m: 
    * alfa, beta, gamma are weights
    * beta (positive feedback) > gamma (negative feedback)
    * user assessed many documents? -> beta, gamma should be high

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) 
- Firstly, run the below code.  
- Observe the results.  
- Assume that we do not like the first and the second result (Docs 63 and 77).  
- However, assume that docs 29 and 36 are satisfactory. Now, modfify the method.  
- It should alter the query vector, according to Rocchio algorithm.   
- Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77).   
- Check the results for different values of alpha, beta, and gamma coefficients.  

In [685]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    
    #####
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    print(ranks)
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

[[<__main__.Document object at 0x7f5bc723c640>, 0.008544060298746436], [<__main__.Document object at 0x7f5bc723c6d0>, 0.0033796012803712916], [<__main__.Document object at 0x7f5bc723ca90>, 0.005693313880880901], [<__main__.Document object at 0x7f5bc723c130>, 0.007849133516984574], [<__main__.Document object at 0x7f5bc723c0d0>, 0.004857394975292029], [<__main__.Document object at 0x7f5bc723c100>, 0.0037797308530653773], [<__main__.Document object at 0x7f5bc723cdf0>, 0.004417028405588663], [<__main__.Document object at 0x7f5bc723cbe0>, 0.009217502327959592], [<__main__.Document object at 0x7f5bc723cd60>, 0.002286550969154977], [<__main__.Document object at 0x7f5bc723c790>, 0.00402377841247803], [<__main__.Document object at 0x7f5bc723c460>, 0.0], [<__main__.Document object at 0x7f5bc7385250>, 0.0027809891929297947], [<__main__.Document object at 0x7f5bc7385790>, 0.006605595469535874], [<__main__.Document object at 0x7f5bc7385550>, 0.008129643070527215], [<__main__.Document object at 0x7f

# 2.3) WordNet (for 5.0)

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [686]:
from nltk.corpus import wordnet as wn

Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [687]:
wn.synsets('machine')

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/home/kasia/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


2.3.3) Display all definitions (.definition()) for synsets (machine)

In [ ]:
#TODO

2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [ ]:
#TODO

See: http://www.nltk.org/howto/wordnet.html
for more examples